1. Dataset Choice

For this lab, we choose MNIST because:

MNIST is small enough to train quickly, even on modest hardware.

It is a common benchmark for adversarial attack research.

Security concerns around digit recognition (e.g., check processing, secure entry systems) can illustrate the impact of adversarial attacks.

Number of images used: We will use the standard MNIST dataset of 60,000 training images and 10,000 test images. From the 60,000 training images, we will create separate train, validation, and test splits.

2. Data Splits & Setup

We will demonstrate two different splitting strategies:

70/15/15 for training, validation, and testing

60/20/20 for training, validation, and testing

The PyTorch torchvision.datasets.MNIST dataset is already split into 60k training and 10k test images, so we will:

Split the 60k “training” images into train and validation based on the desired ratio.

Use the official 10k test images as our “testing” set.

Alternatively, to strictly follow the instructions, you can shuffle the entire 70k images and do your own 70/15/15 or 60/20/20.

Below is an example approach using the official 10k as a base test set, and adjusting the training split internally.

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
import copy
import random


In [14]:
# For reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# Hyperparameters
BATCH_SIZE = 64
EPOCHS = 5
LR = 0.01

# MNIST dataset and transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    # normalizing [0,1] data is optional since MNIST is grayscale (0 to 1),
    # but we could do transforms.Normalize((0.1307,), (0.3081,)) if desired
])

train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)




In [15]:
# Let's define a helper function to create data samplers for 70/15/15 and 60/20/20
def create_data_loaders(train_dataset, test_dataset, train_ratio, val_ratio):
    """
    train_ratio + val_ratio + test_ratio = 1.0 (conceptually)
    We use the official test_dataset as a separate test set.
    """
    num_train = len(train_dataset)  # 60,000
    indices = list(range(num_train))
    np.random.shuffle(indices)

    split_train = int(np.floor(train_ratio * num_train))
    split_val = int(np.floor((train_ratio + val_ratio) * num_train))

    train_idx = indices[:split_train]
    val_idx = indices[split_train:split_val]

    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
    val_loader   = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)
    test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    return train_loader, val_loader, test_loader

In [16]:
# Example for 70/15/15
train_loader_70, val_loader_70, test_loader_70 = create_data_loaders(train_dataset, test_dataset,
                                                                     train_ratio=0.70,
                                                                     val_ratio=0.15)
# Example for 60/20/20
train_loader_60, val_loader_60, test_loader_60 = create_data_loaders(train_dataset, test_dataset,
                                                                     train_ratio=0.60,
                                                                     val_ratio=0.20)

3. Baseline CNN (2 Layers)

3.1 Model Architecture <a name="model-architecture-2-layer"></a>

We will create a simple 2-layer CNN (two convolutional layers followed by fully connected layers). Per the assignment, we cannot use advanced external libraries for the CNN. We will use standard PyTorch modules.

In [17]:
class SimpleCNN_2Layer(nn.Module):
    def __init__(self):
        super(SimpleCNN_2Layer, self).__init__()
        # 1) First conv layer: from 1 channel (grayscale) to 16 channels
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        # 2) Second conv layer: from 16 channels to 32 channels
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)

        # Flatten, then fully connected
        self.fc1 = nn.Linear(32 * 7 * 7, 128)  # after pooling
        self.fc2 = nn.Linear(128, 10)          # 10 classes for MNIST

        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        # Convolution + ReLU + Pool
        x = self.pool(self.relu(self.conv1(x)))  # -> (16, 14, 14)
        x = self.pool(self.relu(self.conv2(x)))  # -> (32, 7, 7)

        # Flatten
        x = x.view(x.size(0), -1)                # -> 32*7*7
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

3.2 Training & Validation <a name="training-and-validation-2-layer"></a>

Below is a generic training loop that we can reuse. We’ll define a helper function train_and_validate(model, train_loader, val_loader, epochs, lr) to train the model and return its best version based on validation accuracy.

In [18]:
def train_and_validate(model, train_loader, val_loader, epochs=5, lr=0.01):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    best_val_acc = 0.0
    best_model_weights = copy.deepcopy(model.state_dict())

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / total
        train_acc = correct / total

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                val_correct += (predicted == labels).sum().item()
                val_total += labels.size(0)
        val_acc = val_correct / val_total

        # Track best weights
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_weights = copy.deepcopy(model.state_dict())

        print(f"Epoch [{epoch+1}/{epochs}] "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
              f"Val Acc: {val_acc:.4f}")

    # Load best weights
    model.load_state_dict(best_model_weights)
    return model


Training on 70/15/15


In [ ]:
model_2layer_70 = SimpleCNN_2Layer()
model_2layer_70 = train_and_validate(model_2layer_70, train_loader_70, val_loader_70, epochs=EPOCHS, lr=LR)


Epoch [1/5] Train Loss: 0.4708, Train Acc: 0.8518, Val Acc: 0.9613


Training on 60/20/20

In [ ]:
model_2layer_60 = SimpleCNN_2Layer()
model_2layer_60 = train_and_validate(model_2layer_60, train_loader_60, val_loader_60, epochs=EPOCHS, lr=LR)

Epoch [1/5] Train Loss: 0.5076, Train Acc: 0.8343, Val Acc: 0.9606
Epoch [2/5] Train Loss: 0.0991, Train Acc: 0.9697, Val Acc: 0.9741
Epoch [3/5] Train Loss: 0.0665, Train Acc: 0.9792, Val Acc: 0.9799
Epoch [4/5] Train Loss: 0.0494, Train Acc: 0.9849, Val Acc: 0.9778


KeyboardInterrupt: 

3.3 Evaluation Metrics <a name="evaluation-metrics-2-layer"></a>

We compute accuracy, precision, recall, F1-score, and AUC-ROC. Note that AUC-ROC for multi-class can be computed in various ways (e.g., one-vs-rest). Below is a simple example using sklearn.metrics.roc_auc_score in a one-vs-rest manner:

In [ ]:
def evaluate_model(model, data_loader):
    model.eval()
    all_labels = []
    all_preds = []
    all_probs = []

    with torch.no_grad():
        for images, labels in data_loader:
            outputs = model(images)
            probs = F.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)

            all_labels.extend(labels.numpy())
            all_preds.extend(predicted.numpy())
            all_probs.extend(probs.numpy())

    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds, average='macro')
    rec = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    # For multi-class AUC, use one-vs-rest approach
    # This requires a one-hot representation of the labels
    all_labels_one_hot = np.zeros((len(all_labels), 10))
    for i, label in enumerate(all_labels):
        all_labels_one_hot[i, label] = 1
    try:
        auc_roc = roc_auc_score(all_labels_one_hot, np.array(all_probs), multi_class="ovr")
    except ValueError:
        # In case some class doesn't appear, fallback to None
        auc_roc = None

    return acc, prec, rec, f1, auc_roc

# Evaluate the two models:
print("=== Evaluation (70/15/15) ===")
acc_70, prec_70, rec_70, f1_70, auc_70 = evaluate_model(model_2layer_70, test_loader_70)
print(f"Accuracy: {acc_70:.4f}, Precision: {prec_70:.4f}, Recall: {rec_70:.4f}, F1: {f1_70:.4f}, AUC-ROC: {auc_70}")

print("\n=== Evaluation (60/20/20) ===")
acc_60, prec_60, rec_60, f1_60, auc_60 = evaluate_model(model_2layer_60, test_loader_60)
print(f"Accuracy: {acc_60:.4f}, Precision: {prec_60:.4f}, Recall: {rec_60:.4f}, F1: {f1_60:.4f}, AUC-ROC: {auc_60}")


3.4 Comparison Table <a name="comparison-table"></a>

2-layer CNN (70/15/15):

    Accuracy:

    Precision:

    Recall:

    F1:

    AUC-ROC:

2-layer CNN (60/20/20):

    Accuracy:

    Precision:

    Recall:

    F1:

    AUC-ROC:


4. Bonus: 3-Layer CNN

We extend the CNN to 3 convolutional layers. For instance:

In [ ]:
class SimpleCNN_3Layer(nn.Module):
    def __init__(self):
        super(SimpleCNN_3Layer, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(2,2)
        self.relu = nn.ReLU()

        self.fc1 = nn.Linear(64*3*3, 128)  # because after 3 conv+pool, size is 3x3
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x))) # -> (16, 14, 14)
        x = self.pool(self.relu(self.conv2(x))) # -> (32, 7, 7)
        x = self.pool(self.relu(self.conv3(x))) # -> (64, 3, 3)
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
model_3layer_70 = SimpleCNN_3Layer()
model_3layer_70 = train_and_validate(model_3layer_70, train_loader_70, val_loader_70, epochs=EPOCHS, lr=LR)

acc_3_70, prec_3_70, rec_3_70, f1_3_70, auc_3_70 = evaluate_model(model_3layer_70, test_loader_70)
print(f"3-Layer CNN (70/15/15) => Accuracy: {acc_3_70:.4f}, Precision: {prec_3_70:.4f}, Recall: {rec_3_70:.4f}, F1: {f1_3_70:.4f}, AUC-ROC: {auc_3_70}")


5. Evasion Attacks (FGSM & PGD)

We now generate adversarial examples to fool our trained model.

5.1 Fast Gradient Sign Method (FGSM)

In [ ]:
def fgsm_attack(model, images, labels, epsilon=0.2):
    # Make images require gradient
    images.requires_grad = True

    outputs = model(images)
    loss = nn.CrossEntropyLoss()(outputs, labels)
    model.zero_grad()
    loss.backward()

    # Collect sign of gradient
    sign_data_grad = images.grad.data.sign()

    # Create adversarial image
    adv_images = images + epsilon * sign_data_grad
    # Clip to [0,1] for valid image
    adv_images = torch.clamp(adv_images, 0, 1)

    return adv_images


Generate adversarial examples on the test set and measure accuracy:

In [ ]:
def test_fgsm(model, device, data_loader, epsilon):
    correct = 0
    adv_examples = []

    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)
        adv_images = fgsm_attack(model, images, labels, epsilon=epsilon)
        outputs = model(adv_images)
        _, final_preds = torch.max(outputs, 1)
        correct += (final_preds == labels).sum().item()

        # Save some examples for visualization
        adv_examples.append((images[0].cpu(), adv_images[0].cpu(), labels[0].cpu(), final_preds[0].cpu()))

    final_acc = correct / len(data_loader.dataset)
    return final_acc, adv_examples

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_2layer_70.to(device)

epsilon = 0.2
acc_fgsm, examples_fgsm = test_fgsm(model_2layer_70, device, test_loader_70, epsilon)
print(f"FGSM Attack with epsilon={epsilon}, Test Accuracy = {acc_fgsm:.4f}")

5.2 Projected Gradient Descent (PGD)

PGD is an iterative version of FGSM. At each iteration, we do a small FGSM step and project back into the ϵ-ball.

In [ ]:
def pgd_attack(model, images, labels, epsilon=0.2, alpha=0.01, iters=10):
    ori_images = images.clone().detach()

    for i in range(iters):
        images.requires_grad = True
        outputs = model(images)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        model.zero_grad()
        loss.backward()
        
        with torch.no_grad():
            adv_images = images + alpha * images.grad.sign()
            # Projection: keep within epsilon-ball of original
            eta = torch.clamp(adv_images - ori_images, min=-epsilon, max=epsilon)
            images = torch.clamp(ori_images + eta, 0, 1).detach_()

    return images

def test_pgd(model, device, data_loader, epsilon, alpha=0.01, iters=10):
    correct = 0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)
        adv_images = pgd_attack(model, images, labels, epsilon=epsilon, alpha=alpha, iters=iters)
        outputs = model(adv_images)
        _, final_preds = torch.max(outputs, 1)
        correct += (final_preds == labels).sum().item()
    final_acc = correct / len(data_loader.dataset)
    return final_acc

acc_pgd = test_pgd(model_2layer_70, device, test_loader_70, epsilon=0.2, alpha=0.01, iters=10)
print(f"PGD Attack, Test Accuracy = {acc_pgd:.4f}")


5.3 Visualizing Adversarial Examples & Accuracy Drop

How the images look before/after attack:

In [ ]:
def visualize_adv_examples(examples, title="FGSM Examples"):
    plt.figure(figsize=(8,10))
    for i in range(6):
        original, adv, label, pred = examples[i]
        plt.subplot(3,4,2*i+1)
        plt.title(f"Original (Label={label})")
        plt.imshow(original.squeeze(), cmap="gray")
        plt.axis('off')

        plt.subplot(3,4,2*i+2)
        plt.title(f"Adversarial (Pred={pred})")
        plt.imshow(adv.squeeze(), cmap="gray")
        plt.axis('off')
    plt.suptitle(title)
    plt.show()

visualize_adv_examples(examples_fgsm, title=f"FGSM e={epsilon}")

We observe  the drop in accuracy from the clean model (e.g., ~98-99%) to the attacked model. Also, you can experiment with varying ϵ to see the trade-off between attack strength (lower accuracy) and detectability (bigger visual perturbations).



6. Data Poisoning Attacks

Data poisoning modifies the training data to degrade or manipulate model behavior. An example is label flipping: choose a fraction of the training samples and flip their labels.

In [ ]:
def poison_labels(dataset, poison_ratio=0.1, flip_from=0, flip_to=1):
    """
    Flip 'flip_from' to 'flip_to' in 'poison_ratio' fraction of the dataset.
    This is a simple example for demonstration.
    """
    n_poison = int(len(dataset) * poison_ratio)
    indices = np.random.choice(len(dataset), size=n_poison, replace=False)
    for idx in indices:
        img, label = dataset[idx]
        if label == flip_from:
            # Replace the label
            dataset[idx] = (img, flip_to)

# Poison 10% of training data: label "0" => label "1"
train_dataset_poisoned = copy.deepcopy(train_dataset)
poison_labels(train_dataset_poisoned, poison_ratio=0.1, flip_from=0, flip_to=1)

# Now train a new model on the poisoned dataset (70/15/15 for example)
train_loader_poisoned_70, val_loader_poisoned_70, test_loader_poisoned_70 = create_data_loaders(
    train_dataset_poisoned, test_dataset, 0.70, 0.15
)

model_poisoned = SimpleCNN_2Layer()
model_poisoned = train_and_validate(model_poisoned, train_loader_poisoned_70, val_loader_poisoned_70,
                                    epochs=EPOCHS, lr=LR)

acc_poisoned, prec_poisoned, rec_poisoned, f1_poisoned, auc_poisoned = evaluate_model(model_poisoned, test_loader_poisoned_70)
print(f"Poisoned Model => Accuracy: {acc_poisoned:.4f}, Precision: {prec_poisoned:.4f}, Recall: {rec_poisoned:.4f}, F1: {f1_poisoned:.4f}, AUC-ROC: {auc_poisoned}")


We observe how the poisoning may degrade overall accuracy or specifically impact digit “0.”

7. Defenses Against Adversarial Attacks

7.1 Adversarial Training

One approach is to retrain the model using a mixture of clean and adversarial examples. During each training iteration, generate adversarial examples (e.g., FGSM) on the fly and include them in the training batch. This helps the model learn robust features.

In [ ]:
def adversarial_training(model, train_loader, epochs=5, lr=0.01, epsilon=0.2):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    for epoch in range(epochs):
        model.train()
        for images, labels in train_loader:
            # Standard forward
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Now generate adversarial examples (FGSM)
            images_adv = fgsm_attack(model, images, labels, epsilon=epsilon)
            outputs_adv = model(images_adv)
            loss_adv = criterion(outputs_adv, labels)

            # Combine losses
            total_loss = loss + loss_adv
            total_loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss.item():.4f}")

    return model

# Example usage:
model_adv_trained = SimpleCNN_2Layer()
model_adv_trained = adversarial_training(model_adv_trained, train_loader_70, epochs=EPOCHS, lr=LR, epsilon=0.2)

# Evaluate against FGSM & PGD now
acc_fgsm_adv, _ = test_fgsm(model_adv_trained, device, test_loader_70, epsilon=0.2)
acc_pgd_adv = test_pgd(model_adv_trained, device, test_loader_70, epsilon=0.2, alpha=0.01, iters=10)
print(f"Adversarially Trained Model => FGSM Acc: {acc_fgsm_adv:.4f}, PGD Acc: {acc_pgd_adv:.4f}")

The adversarially trained model is more robust to FGSM/PGD than the original.

7.2 Feature Squeezing

Feature squeezing reduces the complexity of inputs. Common approaches:

    Bit depth reduction: E.g., round pixel values to fewer bits.
    
    Median filtering or other smoothing.

In [ ]:
def reduce_bit_depth(img, bits=4):
    # img in [0,1], multiply by 255, round, re-scale
    scale_factor = 2 ** bits - 1
    img = torch.round(img * scale_factor) / scale_factor
    return img

def feature_squeezing_inference(model, data_loader, bits=4):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            squeezed = reduce_bit_depth(images, bits)
            outputs = model(squeezed)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return correct / total

acc_squeezed = feature_squeezing_inference(model_2layer_70, test_loader_70, bits=4)
print(f"Feature Squeezing (4-bit) Accuracy: {acc_squeezed:.4f}")

7.3 Differential Privacy

To protect against model inversion attacks, you can train with differential privacy, which injects noise into the gradients or parameters. A simple demonstration could be:

In [ ]:
def differentially_private_training(model, train_loader, epochs=5, lr=0.01, noise_multiplier=1.0):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    for epoch in range(epochs):
        model.train()
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()

            # Add Gaussian noise to gradients for differential privacy
            for param in model.parameters():
                if param.grad is not None:
                    noise = torch.randn_like(param.grad) * noise_multiplier
                    param.grad += noise

            optimizer.step()

        print(f"DP Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

    return model

# Example usage:
model_dp = SimpleCNN_2Layer()
model_dp = differentially_private_training(model_dp, train_loader_70, epochs=EPOCHS, lr=LR, noise_multiplier=0.1)

dp_acc, dp_prec, dp_rec, dp_f1, dp_auc = evaluate_model(model_dp, test_loader_70)
print(f"Differentially Private Model => Accuracy: {dp_acc:.4f}, Precision: {dp_prec:.4f}, "
      f"Recall: {dp_rec:.4f}, F1: {dp_f1:.4f}, AUC-ROC: {dp_auc}")

8. Conclusion

In this lab, we:

Chose the MNIST dataset (justification: widely used, easy to train, and relevant security implications).

Implemented a 2-layer CNN baseline, evaluated on 70/15/15 and 60/20/20 splits, and compared performance metrics.

Extended to a 3-layer CNN for a bonus point.

Demonstrated evasion attacks (FGSM & PGD) and measured how accuracy drops with stronger attacks.

Explored a data poisoning attack by flipping labels in part of the training set.

Implemented and tested defense strategies: Adversarial Training, Feature Squeezing, and Differential Privacy.

Through these exercises, we observed:

How small perturbations can severely degrade model performance in evasion attacks.

How data poisoning can shift model behavior.

How defenses can help, but often at a cost (e.g., performance or training complexity).